1 Year Data

1. 5args
2. 7args
3. 加大盤
4. 加買超賣超


*   5/20
*   10/20
*   20/10







Libraries

In [ ]:
import pandas as pd
import numpy as np
import joblib
import math
import os
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
from google.colab import drive

drive.mount('/content/drive')

FIVE_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume']
SIX_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume', 'II']
SEVEN_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume', 'SMA', 'BIAS']
EIGHT_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume', 'SMA', 'BIAS', 'II']

home = '/content/drive/MyDrive/find_optimal/'
params_dir = 'final_model_params'
one_year_path = home + '0050-KY/'
all_year_path = home + '0050-K/'

SMA_num = 20

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read final model csv and initialize directory


In [ ]:
company_list = ['智邦', '中信金', '統一超', '玉山金', '鴻海', '日月光投控', '亞德客-KY', '瑞昱', '中鋼', '聯發科', '大立光', '南亞科', '台泥', '緯創', '兆豐金', '合庫金', '和碩', '第一金', '遠傳', '豐泰', '富邦金', '台積電', '永豐金', '元大金', '台塑', '欣興', '開發金', '和泰車', '統一', '長榮', '緯穎', '廣達', '上海商銀', '台塑化', '南亞', '國巨', '中租-KY', '華南金', '彰銀', '台新金', '國泰金', '研華', '華碩', '聯詠', '台化', '台灣大', '中華電', '光寶科', '聯電', '台達電']
company_list = company_list[10:20]

final_model = pd.read_csv(home + 'final_model.csv')
final_model.columns = ['company', 'input', 'args', 'mse', 'model', 'days' , 'comment1', 'start', 'comment2']
final_model = final_model.drop(['mse', 'comment1', 'comment2'], axis=1).loc[1:, :].reset_index(drop=True)
print(final_model)

if params_dir not in os.listdir(home):
  os.mkdir(home+params_dir)

for company in company_list:
  if company not in os.listdir(home+params_dir):
    os.mkdir(home+params_dir+'/'+company)
  for f in os.listdir(home+params_dir+'/'+company):
    os.remove(home+params_dir+'/'+company+'/'+f)

params_path = home + params_dir + '/'



   company input args model    days start
0       智邦    20    5     7     all   NaN
1      中信金    10    6     4     all   NaN
2      大立光     5    5     4     one   NaN
3    日月光投控     5    7     1     one   NaN
4      統一超     5    8     6     one   NaN
5      南亞科    10    8     4     one   NaN
6   亞德客-KY     5    7     7  select  2500
7       鴻海     5    6     6     one  3500
8       中鋼    20    7     4     one   NaN
9      聯發科    10    5     5     all   NaN
10      瑞昱     5    7     4  select  4500
11      緯創    20    5     5     all   NaN
12     兆豐金    20    7     7     one  4000
13     國泰金     5    7     1     one   NaN
14      台泥    10    5     5     all   NaN
15      聯詠    20    7     1     one   NaN
16      台化     5    5     1     one   NaN
17      華碩     5    5     4     all   NaN
18      研華    20    6     4     one   NaN
19     玉山金     5    6     1     one   NaN
20      緯穎    20    8     1     one   NaN
21      廣達    20    5     5     all  3000
22      南亞    10    6     1     on

Input CSV File

In [ ]:
company_df_dict = {}
for company in company_list:
  # print(company)
  if final_model.loc[final_model['company'] == company, 'days'].values[0] == 'one':
    df = pd.read_csv(one_year_path+company+'/K.csv')
    df = df.iloc[2:, 1:16]
    df = df.drop(['漲跌','漲跌.1','振幅','Unnamed: 9','Unnamed: 10','Unnamed: 11','法人買賣超(張)','Unnamed: 13','Unnamed: 14'],axis=1)

    df.columns = ['Open','High','Low','Close','Volume','II']
    df['Volume'] = df['Volume'].apply(lambda x: float(str(x).replace(',','')))
    df['II'] = df['II'].apply(lambda x: float(str(x).replace(',','')))
    df = df.iloc[::-1].reset_index(drop=True)
    if int(final_model.loc[final_model['company'] == company, 'args'].values[0]) == 5 or int(final_model.loc[final_model['company'] == company, 'args'].values[0]) == 7:
      df = df.drop(['II'], axis=1)

  else:
    df = pd.read_csv(all_year_path+company+'/K.csv')
    df = df.iloc[:, 1:]
    df = df.drop(['Adj Close'],axis=1)
    df = df.dropna()
    df = df.loc[df['Volume'] != 0.0].reset_index(drop=True)

  if int(final_model.loc[final_model['company'] == company, 'args'].values[0]) >= 7:
    SMA = []
    BIAS = []
    for i in range(len(df)):
      if i < SMA_num - 1:
        SMA.append(0)
        BIAS.append(0)
      else:
        SMA.append(df.loc[i+1-SMA_num:i+1, 'Open'].mean())
        BIAS.append((df.loc[i, 'Open'] - SMA[i]) / SMA[i])
    df['SMA'] = SMA
    df['BIAS'] = BIAS
    df = df.loc[SMA_num:].reset_index(drop=True)

  if final_model.loc[final_model['company'] == company, 'days'].values[0] == 'select':
    df = df[int(final_model.loc[final_model['company'] == company, 'start'].values[0]):].reset_index(drop=True)

  # print(df.head())
  company_df_dict[company] = df

Functions

In [ ]:
def Split_Data(df, input_num, predict_num, split_point):
  train_days = split_point-input_num+1
  X_train = []
  Y_train = []

  for i in range(train_days):
    X_train.append(df.loc[i:i+input_num-1, :].values)
    Y_train.append([df.loc[i+input_num+predict_num-1, 'Open']])

  return np.array(X_train), np.array(Y_train)

def Build_Model(X_train, Y_train, model_num):
  model = Sequential()
  if model_num == 1:
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(LSTM(50))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=64, epochs=100)
  elif model_num == 2:
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(LSTM(50))
    model.add(Dense(25))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=64, epochs=100)
  elif model_num == 3:
    model.add(LSTM(30, return_sequences=True, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(LSTM(30))
    model.add(Dense(15))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=64, epochs=100)
  elif model_num == 4:
    model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=16, epochs=100, verbose=1)
  elif model_num == 5:
    model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=32, epochs=100)
  elif model_num == 6:
    model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64))
    model.add(Dense(25))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=32, epochs=100)
  elif model_num == 7:
    model.add(LSTM(30, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(30))
    model.add(Dense(15))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, Y_train, batch_size=32, epochs=100)

  return model

Main

In [ ]:
def predict(model, X, scaler_open):

  future_predictions = model.predict(X)
  future_predictions = scaler_open.inverse_transform(future_predictions)
  future_predictions = future_predictions.reshape(future_predictions.shape[1]*future_predictions.shape[0])

  return future_predictions

In [ ]:
for company, df_comp in tqdm(company_df_dict.items()):
  print('\n'+company+'\n')

  input_num = int(final_model.loc[final_model['company'] == company, 'input'])
  model_num = int(final_model.loc[final_model['company'] == company, 'model'])

  params_path = home + 'final_model_params/'
  # normalization and save scaler
  scaler_open, scaler = None, None
  # for col in df_comp.columns:
  #   scaler = MinMaxScaler(feature_range=(0,1))
  #   df_comp[col] = scaler.fit_transform(df_comp.filter([col]).values)
  #   joblib.dump(scaler, params_path + company + '/' + f'scaler_{col}.z')
  scalers = {}
  for col in df_comp.columns:
      scaler = joblib.load(params_path + company + '/' + f'scaler_{col}.z')
      scalers[col] = scaler

  model = load_model(params_path + company + '/model.keras')
  for col in df_comp.columns:
      if col in scalers:
          df_comp[col] = scalers[col].transform(df_comp.filter([col]).values)

  predictions = predict(model, df_comp.values, scalers['Open'])

  #  train and save final model
  # split_point = math.ceil(len(df_comp)* .9)
  # X_train, Y_train = Split_Data(df_comp, input_num, 5, split_point)
  # model = Build_Model(X_train, Y_train, model_num)
  # model.save(params_path + company + '/model.keras')

  0%|          | 0/10 [00:00<?, ?it/s]


大立光



FileNotFoundError: ignored